In [94]:
#pip install selenium==4.9.0

In [13]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd
from groq import Groq
import sqlite3

import time

url = "https://item.rakuten.co.jp/tansu/43500002/"

def move_to_reviews(url):
    # driver = webdriver.Chrome(executable_path="chromedriver.exe")
     
    chrome_options = Options()
    
    
    chrome_options.add_argument("--headless")
    driver = webdriver.Chrome(options=chrome_options)
    
    
    driver.get(f"{url}")
    
    WebDriverWait(driver, 2).until(
        EC.presence_of_all_elements_located((By.PARTIAL_LINK_TEXT, "件）")))
    
    link_element = driver.find_element(By.PARTIAL_LINK_TEXT, "件）")
    link_url = link_element.get_attribute('href').split('1.1')[0]

    time.sleep(2)
    driver.quit()

    return link_url

url_link = move_to_reviews(url)


0.0


In [20]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd
from groq import Groq
import sqlite3

import time

def move_to_reviews(url):
    start_time = time.time()

    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--disable-extensions")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--disable-images")
    chrome_options.add_argument("--disable-javascript")
    chrome_options.add_argument("--disable-plugins")
    chrome_options.add_argument("--enable-fast-unload")
    chrome_options.add_argument("--disable-popup-blocking")
    chrome_options.add_argument("--disable-notifications")
    chrome_options.add_argument("--disable-background-networking")
    chrome_options.add_argument("--disable-background-timer-throttling")
    chrome_options.add_argument("--disable-backgrounding-occluded-windows")
    chrome_options.add_argument("--disable-client-side-phishing-detection")
    chrome_options.add_argument("--disable-hang-monitor")
    chrome_options.add_argument("--disable-prompt-on-repost")
    chrome_options.add_argument("--disable-sync")
    chrome_options.add_argument("--disable-translate")
    chrome_options.add_argument("--metrics-recording-only")
    chrome_options.add_argument("--safebrowsing-disable-auto-update")
    chrome_options.add_argument("--disable-voice-input")

    # Disable images
    prefs = {"profile.managed_default_content_settings.images": 2}
    chrome_options.add_experimental_option("prefs", prefs)
    
    driver = webdriver.Chrome(options=chrome_options)

    init_driver_time = time.time()
    print(f"Time to initialize driver: {init_driver_time - start_time} seconds")

    driver.get(f"{url}")

    navigate_url_time = time.time()
    print(f"Time to navigate to URL: {navigate_url_time - init_driver_time} seconds")

    WebDriverWait(driver, 2).until(
        EC.presence_of_all_elements_located((By.PARTIAL_LINK_TEXT, "件）"))
    )

    wait_for_element_time = time.time()
    print(f"Time to wait for element: {wait_for_element_time - navigate_url_time} seconds")

    link_element = driver.find_element(By.PARTIAL_LINK_TEXT, "件）")
    find_element_time = time.time()
    print(f"Time to find element: {find_element_time - wait_for_element_time} seconds")

    link_url = link_element.get_attribute('href').split('1.1')[0]
    get_attribute_time = time.time()
    print(f"Time to get attribute and process URL: {get_attribute_time - find_element_time} seconds")

    time.sleep(2)
    sleep_time = time.time()
    print(f"Time to sleep for 2 seconds: {sleep_time - get_attribute_time} seconds")

    driver.quit()
    quit_driver_time = time.time()
    print(f"Time to quit driver: {quit_driver_time - sleep_time} seconds")

    print(f"Total time to run function: {quit_driver_time - start_time} seconds")

    return link_url

# Example usage
url_link = move_to_reviews(url)


Time to initialize driver: 1.0378897190093994 seconds
Time to navigate to URL: 16.163315057754517 seconds
Time to wait for element: 0.038630008697509766 seconds
Time to find element: 0.02902674674987793 seconds
Time to get attribute and process URL: 0.013146638870239258 seconds
Time to sleep for 2 seconds: 2.0012168884277344 seconds
Time to quit driver: 3.9014110565185547 seconds
Total time to run function: 23.184636116027832 seconds


In [21]:
def get_soup(link_url):
    response = requests.get(link_url)
    soup = BeautifulSoup(response.text, "html.parser")
    return soup


def product_details(soup):
    total_reviews = soup.find("span", {"class": "Count"}).text.strip()
    price = soup.find("span", {"revPrice"}).text.strip()
    product = soup.find("h2", {"class": "revItemTtl"}).text.strip()
    img_tag = soup.select_one('#thumbWindow img')
    if img_tag:
        image_link = img_tag.get('src')

    details = {
        'product_name': product,
        'product_image': image_link,
        'product_price': price + '円',
        'total_reviews': total_reviews
    }

    return details

k = get_soup(url_link)
y = product_details(k)
y 

{'product_name': 'グーン まっさらさら通気／ぐんぐん吸収 パンツ ディズニー(3個)【mmw】【グーン(GOO.N)】[オムツ 紙おむつ 紙オムツ]',
 'product_image': 'https://thumbnail.image.rakuten.co.jp/@0_mall/netbaby/cabinet/196/404196.jpg?_ex=180x180',
 'product_price': '4,899円',
 'total_reviews': '2,610'}

In [17]:
def page_jumper(num_pages, url_link):
    soup = get_soup(url_link)
    total_reviews = int(soup.find("span", {"class": "Count"}).text.strip().replace(',', ''))

    total_pages = int(total_reviews)/15
    pages = num_pages

    link_list = []
    if pages < total_pages:
        for i in range(pages):
            link_list.append(f"{url_link}{i+1}.1/")
        return link_list
    else:
        return "Number of pages doesn't exist"

links = page_jumper(11, url_link)
print(links)

['https://review.rakuten.co.jp/item/1/212232_10027922/1.1/', 'https://review.rakuten.co.jp/item/1/212232_10027922/2.1/', 'https://review.rakuten.co.jp/item/1/212232_10027922/3.1/', 'https://review.rakuten.co.jp/item/1/212232_10027922/4.1/', 'https://review.rakuten.co.jp/item/1/212232_10027922/5.1/', 'https://review.rakuten.co.jp/item/1/212232_10027922/6.1/', 'https://review.rakuten.co.jp/item/1/212232_10027922/7.1/', 'https://review.rakuten.co.jp/item/1/212232_10027922/8.1/', 'https://review.rakuten.co.jp/item/1/212232_10027922/9.1/', 'https://review.rakuten.co.jp/item/1/212232_10027922/10.1/', 'https://review.rakuten.co.jp/item/1/212232_10027922/11.1/']


In [4]:
def product_reviews(link_list):
    reviewlist = []
    
    for link in link_list:
        soup = get_soup(link)
        reviews = soup.find_all("div", {"class": "revRvwUserSec"})
        
        for item in reviews:
            review = {
            'rating': float(item.find("span", {"class":"revUserRvwerNum"}).text.strip()),
            'date': item.find("span", {"class":"revUserEntryDate"}).text.strip(),
            'body': item.find("dd", {"class":"revRvwUserEntryCmt"}).text.strip(),
            }
            reviewlist.append(review)
    return reviewlist
    

reviews = product_reviews(links)

In [5]:
def convert_to_df(reviews):
    df_reviews = pd.DataFrame(reviews)
    descriptions = df_reviews['body']
    return descriptions

desc = convert_to_df(reviews)
print(desc)

0      前回の期間限定トイストーリー柄良かったな。ずっとＭサイズでプーさん柄で、Lにサイズアップして...
1      20％ポイントバックがある楽天セール期間中に、500円OFFクーポンを使ってお得に購入した。...
2      前回の期間限定トイストーリー柄良かったな。ずっとＭサイズでプーさん柄で、Lにサイズアップして...
3      1歳3か月（女の子）、体重約9kgです。\nいつも1枚あたりが一番安いオムツを楽天で買ってい...
4      ポリマーの違いなどはわかりませんが、旧製品より肌触りが良くふわふわになったと思います。（試し...
                             ...                        
160           スパセでポイントバックがあり、実質安く買えました！\n届くのも早くて助かりましたー！
161                                 購入から到着までとても早く助かりました。
162                  即日発送して頂けたようで2日で届きました。\nありがとうございました。
163    セールでお得に購入。ポイント還元とか色々考えて計算すると、近所のドラストとかよりもお得だった...
164            ポイントバックがある時に買っています！\n郵送して貰えるし、とても助かっています！
Name: body, Length: 165, dtype: object


In [6]:
import os
def sentiment(descriptions):
    client = Groq(
        api_key=os.environ.get("GROQ_API_KEY"),
    )

    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": f"Analyzing the {descriptions} give me the general sentiment regarding the product and suggestions for how to imporve the product, increase the sales and get better reviews",
            }
        ],
        model="llama3-70b-8192",
    )
    return chat_completion.choices[0].message.content


sentiments = sentiment(desc)
print(sentiments)

**General Sentiment:**

The general sentiment regarding the product is overwhelmingly positive. Customers are satisfied with their purchase, citing benefits such as:

* Good value for money
* Fast delivery
* Comfortable and soft product ( likely a diaper or clothing item)
* Convenience of online shopping
* Benefits of point-back systems and coupons

**Suggestions for Improvement:**

Based on the customer feedback, here are some suggestions for improvement:

1. **Expand Size Options:** One customer mentioned wanting to size up from M to L, implying that offering more size options could be beneficial.
2. **Material Quality:** While customers appreciate the comfort and softness of the product, some may be interested in understanding the differences between materials (e.g., polymers). Providing education on this aspect could enhance customer satisfaction.
3. **Wider Product Range:** Customers may appreciate a broader range of products, as they seem to be interested in purchasing from the s

In [7]:
def convert_to_df(reviews):
    df_reviews = pd.DataFrame(reviews)
    conn = sqlite3.connect('rakuten_reviews.db')
    df_reviews.to_sql('reviews', conn, if_exists='replace', index=False)
    conn.close()
    
def query_db():
    conn = sqlite3.connect('rakuten_reviews.db')
    df_reviews = pd.read_sql_query('SELECT rating, body FROM reviews', conn)
    conn.close()
    return df_reviews

convert_to_df(reviews)
df_from_db = query_db()
print(df_from_db)

     rating                                               body
0       5.0  前回の期間限定トイストーリー柄良かったな。ずっとＭサイズでプーさん柄で、Lにサイズアップして...
1       5.0  20％ポイントバックがある楽天セール期間中に、500円OFFクーポンを使ってお得に購入した。...
2       5.0  前回の期間限定トイストーリー柄良かったな。ずっとＭサイズでプーさん柄で、Lにサイズアップして...
3       5.0  1歳3か月（女の子）、体重約9kgです。\nいつも1枚あたりが一番安いオムツを楽天で買ってい...
4       5.0  ポリマーの違いなどはわかりませんが、旧製品より肌触りが良くふわふわになったと思います。（試し...
..      ...                                                ...
160     5.0         スパセでポイントバックがあり、実質安く買えました！\n届くのも早くて助かりましたー！
161     5.0                               購入から到着までとても早く助かりました。
162     5.0                即日発送して頂けたようで2日で届きました。\nありがとうございました。
163     5.0  セールでお得に購入。ポイント還元とか色々考えて計算すると、近所のドラストとかよりもお得だった...
164     4.0          ポイントバックがある時に買っています！\n郵送して貰えるし、とても助かっています！

[165 rows x 2 columns]


In [ ]:
# def product_reviews(soup):

#     revi`ews = soup.find_all("div", {"class": "revRvwUserSec"})
#     reviewlist = []
#     for item in reviews:
#         review = {
#         'rating': float(item.find("span", {"class":"revUserRvwerNum"}).text.strip()),
#         'date': item.find("span", {"class":"revUserEntryDate"}).text.strip(),
#         'body': item.find("dd", {"class":"revRvwUserEntryCmt"}).text.strip(),
#         }
#         reviewlist.append(review)

#     reviews = {
#         'product_reviews': reviewlist
#     }
#     return reviews

In [79]:
# def getnextpage(soup):
#     page = soup.find("div", {"class":"revPagination"})
    
#     if page.find("div", {"class":"revThisPage"}):
#         url = page.find("div", {"class":"revPagination"}).find('a')['href']
#         return url
#     else:
#         return 


In [4]:
# link_url = move_to_reviews(url)
# soup = get_soup(link_url)
# product = product_details(soup)
# reviews = product_reviews(soup)



In [ ]:
# response = requests.get(url)
# print(response.text)

# headers = {
#     "authority":"item.rakuten.co.jp",
#     "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
#     "Accept-Language":"en-US,en;q=0.9",
#     "Sec-Ch-Ua": '"Opera";v="109", "Not:A-Brand";v="8", "Chromium";v="123"',
#     "User-Agent":"Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Mobile Safari/537.36",  
# }

# url = 'https://item.rakuten.co.jp/kabegamiyahonpo/rkwp-4999set7/'

In [ ]:
class ProductDetailsAndReviews:
    def __init__(self, soup):
        self.soup = soup

    def get_product_details(self):
        product = self.soup.find("h2", {"class": "revItemTtl"}).text.strip()
        img_tag = self.soup.select_one('#thumbWindow img')
        image_link = img_tag.get('src') if img_tag else None

        details = {
            'product_name': product,
            'product_image': image_link,
        }

        return details

    def get_product_reviews(self):
        reviews = self.soup.find_all("div", {"class": "revRvwUserSec"})
        review_list = []
        for item in reviews:
            review = {
                'rating': float(item.find("span", {"class":"revUserRvwerNum"}).text.strip()),
                'date': item.find("span", {"class":"revUserEntryDate"}).text.strip(),
                'body': item.find("dd", {"class":"revRvwUserEntryCmt"}).text.strip(),
            }
            review_list.append(review)

        reviews = {
            'product_reviews': review_list
        }

        return reviews

# Usage
product_details_and_reviews = ProductDetailsAndReviews(soup)
details = product_details_and_reviews.get_product_details()
reviews = product_details_and_reviews.get_product_reviews()
